In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
# RGB scale
train_tape = "C:/rppg/cohface_datatape_128x32x32_train.h5"
valid_tape = "C:/rppg/cohface_datatape_128x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

#train, valid = to_tf(train).cache(f'{tmp}/physnet'), to_tf(valid).cache(f'{tmp}/physnet')
train, valid = to_tf(train), to_tf(valid)

In [3]:
# RGB Scale
phys_net = PhysNet()
phys_net.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss=np_loss)
phys_net.build(input_shape=(None, 128, 32, 32, 3))
print(f'Flops per frame:{get_flops(phys_net, input_sig=[tf.TensorSpec([1, 128, 32, 32, 3])])/128:.0f}')
phys_net.summary()

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Flops per frame:54260867
Model: "phys_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 128, 32, 32, 16)   1280      
                                                                 
 sequential_1 (Sequential)   (None, 128, 16, 16, 32)   13984     
                                                                 
 sequential_2 (Sequential)   (None, 128, 16, 16, 64)   55616     
                                                                 
 sequential_3 (Sequential)   (None, 64, 8, 8, 64)      110912    
                                                                 
 sequential_4 (Sequential)   (None, 64, 8, 8, 64)      110912    
                                                       

In [4]:
phys_net.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=3, callbacks=[keras.callbacks.ModelCheckpoint('../weights/PhysNet_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
phys_net.load_weights('../weights/PhysNet_CCNU.h5')

Epoch 1/3
     85/Unknown - 1151s 13s/step - loss: 0.9667
Epoch 1: val_loss improved from inf to 0.99786, saving model to ../weights\PhysNet_CCNU.h5
85/85 [==============================] - 1247s 15s/step - loss: 0.9667 - val_loss: 0.9979
Epoch 2/3
85/85 [==============================] - ETA: 0s - loss: 0.8551 
Epoch 2: val_loss improved from 0.99786 to 0.99041, saving model to ../weights\PhysNet_CCNU.h5
85/85 [==============================] - 1247s 15s/step - loss: 0.8551 - val_loss: 0.9904
Epoch 3/3
85/85 [==============================] - ETA: 0s - loss: 0.6954 
Epoch 3: val_loss improved from 0.99041 to 0.96438, saving model to ../weights\PhysNet_CCNU.h5
85/85 [==============================] - 1256s 15s/step - loss: 0.6954 - val_loss: 0.9644


In [2]:
# gray scale
train_tape = "C:/rppg/cohface_datatape_gray2_128x32x32_train.h5"
valid_tape = "C:/rppg/cohface_datatape_gray2_128x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

#train, valid = to_tf(train).cache(f'{tmp}/physnet'), to_tf(valid).cache(f'{tmp}/physnet')
train, valid = to_tf(train), to_tf(valid)

In [3]:
# Gray Scale
phys_net = PhysNetGray()
phys_net.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss=np_loss)
phys_net.build(input_shape=(None, 128, 32, 32))
print(f'Flops per frame:{get_flops(phys_net, input_sig=[tf.TensorSpec([1, 128, 32, 32])])/128:.0f}')
phys_net.summary()

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Flops per frame:1766339
Model: "phys_net_gray"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 128, 32, 16)       2640      
                                                                 
 sequential_1 (Sequential)   (None, 128, 16, 32)       4768      
                                                                 
 sequential_2 (Sequential)   (None, 128, 16, 64)       18752     
                                                                 
 sequential_3 (Sequential)   (None, 64, 8, 64)         37184     
                                                                 
 sequential_4 (Sequential)   (None, 64, 8, 64)         37184     
                                                   

In [4]:
# grayscale
phys_net.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=3, callbacks=[keras.callbacks.ModelCheckpoint('../weights/PhysNet_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
phys_net.load_weights('../weights/PhysNet_CCNU.h5')

Epoch 1/3
     85/Unknown - 17s 160ms/step - loss: 0.9890
Epoch 1: val_loss improved from inf to 0.99204, saving model to ../weights\PhysNet_CCNU.h5
85/85 [==============================] - 21s 215ms/step - loss: 0.9890 - val_loss: 0.9920
Epoch 2/3
85/85 [==============================] - ETA: 0s - loss: 0.9539
Epoch 2: val_loss did not improve from 0.99204
85/85 [==============================] - 19s 211ms/step - loss: 0.9539 - val_loss: 0.9921
Epoch 3/3
85/85 [==============================] - ETA: 0s - loss: 0.9096
Epoch 3: val_loss improved from 0.99204 to 0.99194, saving model to ../weights\PhysNet_CCNU.h5
85/85 [==============================] - 19s 214ms/step - loss: 0.9096 - val_loss: 0.9919


In [10]:
phys_net.load_weights('../weights/PhysNet_CCNU.h5')

In [12]:
eval_on_dataset(test_set_CCNU, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_cohface_gray.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_CCNU_cohface.h5')

  0%|                                                                                          | 0/164 [00:00<?, ?it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\asust\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\asust\AppData\Local\Temp\ipykernel_19832\1943848672.py", line 1, in <module>
    eval_on_dataset(test_set_COHFACE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_cohface.h5', sample=cv2.INTER_CUBIC)
  File "C:\Users\asust\Downloads\PhysBench-main\PhysBench-main\benchmark_RLAP\..\utils.py", line 814, in eval_on_dataset
    o = model(np.stack([i[1] for i in ipt[idx:idx+batch]]).astype(ipt_dtype))
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\asust\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\asust\Downloads\PhysBench-main\PhysBench-main\benchmark_RLAP\..\models.py", line 467, in call
    x = 

In [5]:
#eval_on_dataset(test_set_CCNU_rPPG, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_CCNU_rPPG.h5', sample=cv2.INTER_CUBIC)
#get_metrics('../results/PhysNet_CCNU_CCNU_rPPG.h5')
get_metrics_HRV('../results/PhysNet_CCNU_CCNU_rPPG.h5')

{'SDNN': {'MAE': 36.441, 'RMSE': 43.784, 'R': 0.30584}}

In [4]:
#eval_on_dataset(test_set_PURE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_PURE.h5', sample=cv2.INTER_CUBIC)
#get_metrics('../results/PhysNet_CCNU_PURE.h5')
get_metrics_HRV('../results/PhysNet_CCNU_PURE.h5')

x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\ma\core.py:5288: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


{'SDNN': {'MAE': 22.51, 'RMSE': 35.702, 'R': 0.56025}}

In [3]:
#eval_on_dataset(test_set_UBFC_rPPG2, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_UBFC.h5', sample=cv2.INTER_CUBIC)
#get_metrics('../results/PhysNet_CCNU_UBFC.h5')
get_metrics_HRV('../results/PhysNet_CCNU_UBFC.h5')

x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\scipy\interpolate\_fitpack2.py:298: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


{'SDNN': {'MAE': 12.189, 'RMSE': 14.907, 'R': 0.88684}}

In [7]:
#eval_on_dataset(test_set_MMPD, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_MMPD.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_CCNU_MMPD.h5', motion='Stationary', skin_color='3', light=['LED-high', 'LED-low', 'Incandescent'])

{'Sliding window': {'MAE': 1.657, 'RMSE': 4.498, 'R': 0.8773},
 'Whole video': {'MAE': 0.972, 'RMSE': 1.453, 'R': 0.98796}}

In [13]:
eval_on_dataset(test_set_COHFACE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_COHFACE.h5')
get_metrics('../results/PhysNet_CCNU_COHFACE.h5')

  0%|                                                                                          | 0/164 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "C:\Users\asust\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\asust\AppData\Local\Temp\ipykernel_19832\2500310600.py", line 1, in <module>
    eval_on_dataset(test_set_COHFACE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_COHFACE.h5')
  File "C:\Users\asust\Downloads\PhysBench-main\PhysBench-main\benchmark_RLAP\..\utils.py", line 814, in eval_on_dataset
    o = model(np.stack([i[1] for i in ipt[idx:idx+batch]]).astype(ipt_dtype))
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\asust\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\asust\Downloads\PhysBench-main\PhysBench-main\benchmark_RLAP\..\models.py", line 467, in call
    x = self.ConvBlock1(x, trai

In [8]:
eval_on_dataset(test_set_COHFACE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_cohface_gray.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_CCNU_cohface_gray.h5')

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [05:40<00:00,  2.08s/it]


{'Sliding window': {'MAE': 19.608, 'RMSE': 26.239, 'R': -0.257},
 'Whole video': {'MAE': 20.214, 'RMSE': 27.242, 'R': -0.39406}}